In [1]:
# We will create a data generator object here that would be helpful when doing bootstrap samples

# Later it is used for predictions, as we cannot train directly with the notebook for some reason. But we can use a model that is store, and predict the response in the notebook. The main two scripts here with the new generator from the tf.keras.Sequence class are DeepCDR_train_with_new_generator.py (contains training the model with the new data generator), and New_data_generator_with_tf.py which gives the defined functions for the data generator, and the batch wise prediction. We will also include this nb as it contains some rough work in the process.

In [2]:
# we will still need all our data, and we will need to see if indeed our new generator work, that's what we will do here

In [3]:
# let's import the necessary packages here

In [4]:
# These are the ones that is already on the train script
import os
import json
import sys
import warnings
from pathlib import Path
from pprint import pformat
from typing import Dict, Union
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from create_data_generator import data_generator, batch_predict

# [Req] IMPROVE imports
# notice that the improvelibs are in the folder that is a level above, but in the same parent directory
sys.path.append(os.path.abspath(os.path.join('..', 'IMPROVE')))
from improvelib.applications.drug_response_prediction.config import DRPTrainConfig
from improvelib.utils import str2bool
import improvelib.utils as frm
from improvelib.metrics import compute_metrics

# Model-specific imports
from model_params_def import train_params # [Req]

2025-04-16 13:00:25.902576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-16 13:00:26.727285: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
# any new additional packages necessary for the generator goes here
from tensorflow.keras.utils import Sequence
# need numpy here as well, but we already have this

In [6]:
# we need all our data, and possibly the model too, let's make sure we have all these things here

In [7]:
training = False
dropout1 = 0.10
dropout2 = 0.20
## get the model architecture
def deepcdrgcn(dict_features, dict_adj_mat, samp_drug, samp_ach, cancer_dna_methy_model, cancer_gen_expr_model, cancer_gen_mut_model, training = training, dropout1 = dropout1, dropout2 = dropout2):
    
    input_gcn_features = tf.keras.layers.Input(shape = (dict_features[samp_drug].shape[0], 75))
    input_norm_adj_mat = tf.keras.layers.Input(shape = (dict_adj_mat[samp_drug].shape[0], dict_adj_mat[samp_drug].shape[0]))
    mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_1)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)
    mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
    dense_out = dense_layer_gcn(mult_2)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
    mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
    dense_out = dense_layer_gcn(mult_3)
    dense_out = tf.keras.layers.BatchNormalization()(dense_out)
    dense_out = tf.keras.layers.Dropout(dropout1)(dense_out, training = training)

    dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)
    # All above code is for GCN for drugs

    # methylation data
    input_gen_methy1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_methy = cancer_dna_methy_model(input_gen_methy1)
    input_gen_methy.trainable = False
    gen_methy_layer = tf.keras.layers.Dense(256, activation = "tanh")
    
    gen_methy_emb = gen_methy_layer(input_gen_methy)
    gen_methy_emb = tf.keras.layers.BatchNormalization()(gen_methy_emb)
    gen_methy_emb = tf.keras.layers.Dropout(dropout1)(gen_methy_emb, training = training)
    gen_methy_layer = tf.keras.layers.Dense(100, activation = "relu")
    gen_methy_emb = gen_methy_layer(gen_methy_emb)

    # gene expression data
    input_gen_expr1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_expr = cancer_gen_expr_model(input_gen_expr1)
    input_gen_expr.trainable = False
    gen_expr_layer = tf.keras.layers.Dense(256, activation = "tanh")
    
    gen_expr_emb = gen_expr_layer(input_gen_expr)
    gen_expr_emb = tf.keras.layers.BatchNormalization()(gen_expr_emb)
    gen_expr_emb = tf.keras.layers.Dropout(dropout1)(gen_expr_emb, training = training)
    gen_expr_layer = tf.keras.layers.Dense(100, activation = "relu")
    gen_expr_emb = gen_expr_layer(gen_expr_emb)
    
    
    input_gen_mut1 = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    input_gen_mut = cancer_gen_mut_model(input_gen_mut1)
    input_gen_mut.trainable = False
    
    reshape_gen_mut = tf.keras.layers.Reshape((1, cancer_gen_mut_model(samp_ach).numpy().shape[0], 1))
    reshape_gen_mut = reshape_gen_mut(input_gen_mut)
    gen_mut_layer = tf.keras.layers.Conv2D(50, (1, 700), strides=5, activation = "tanh")
    gen_mut_emb = gen_mut_layer(reshape_gen_mut)
    pool_layer = tf.keras.layers.MaxPooling2D((1,5))
    pool_out = pool_layer(gen_mut_emb)
    gen_mut_layer = tf.keras.layers.Conv2D(30, (1, 5), strides=2, activation = "relu")
    gen_mut_emb = gen_mut_layer(pool_out)
    pool_layer = tf.keras.layers.MaxPooling2D((1,10))
    pool_out = pool_layer(gen_mut_emb)
    flatten_layer = tf.keras.layers.Flatten()
    flatten_out = flatten_layer(pool_out)
    x_mut = tf.keras.layers.Dense(100,activation = 'relu')(flatten_out)
    x_mut = tf.keras.layers.Dropout(dropout1)(x_mut)
    
    all_omics = tf.keras.layers.Concatenate()([dense_out, gen_methy_emb, gen_expr_emb, x_mut])
    x = tf.keras.layers.Dense(300,activation = 'tanh')(all_omics)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
    x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=1))(x)
    x = tf.keras.layers.Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,2))(x)
    x = tf.keras.layers.Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
    x = tf.keras.layers.Dropout(dropout1)(x, training = training)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout2)(x, training = training)
    final_out_layer = tf.keras.layers.Dense(1, activation = "linear")
    final_out = final_out_layer(x)
    simplecdr = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat, input_gen_expr1,
                                   input_gen_methy1, input_gen_mut1], final_out)
    
    return simplecdr

In [8]:
modelpath = frm.build_model_path(model_file_name="DeepCDR_model", model_file_format="", model_dir="exp_result")

In [9]:
modelpath

PosixPath('exp_result/DeepCDR_model')

In [10]:
train_data_fname = frm.build_ml_data_file_name(data_format=".csv", stage="train")  # [Req]
val_data_fname = frm.build_ml_data_file_name(data_format=".csv", stage="val")  # [Req]

In [11]:
print(train_data_fname)
print(val_data_fname)

train_data.csv
val_data.csv


In [12]:
# specify the directory where preprocessed data is stored
data_dir = 'exp_result'

In [13]:
%%time
# load the models
cancer_gen_expr_model = tf.keras.models.load_model(os.path.join(data_dir,"cancer_gen_expr_model"))
cancer_gen_mut_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_gen_mut_model"))
cancer_dna_methy_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_dna_methy_model"))

2025-04-16 13:00:44.116444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-16 13:00:46.281705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30960 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


WARNING tensorflow 2025-04-16 13:00:47,241:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-16 13:00:47,525:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-16 13:00:47,759:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


CPU times: user 1.28 s, sys: 895 ms, total: 2.17 s
Wall time: 3.92 s


In [14]:
cancer_gen_expr_model.trainable = False
cancer_gen_mut_model.trainable = False
cancer_dna_methy_model.trainable = False

In [15]:
with open(os.path.join(data_dir, "drug_features.pickle"),"rb") as f:
        dict_features = pickle.load(f)

In [16]:
with open(os.path.join(data_dir, "norm_adj_mat.pickle"),"rb") as f:
        dict_adj_mat = pickle.load(f)

In [17]:
train_keep = pd.read_csv(os.path.join(data_dir, "train_y_data.csv"))
valid_keep = pd.read_csv(os.path.join(data_dir, "val_y_data.csv"))

In [18]:
train_keep.head()

,improve_sample_id,improve_chem_id,auc
0,ACH-000956,Drug_749,0.7153
1,ACH-000956,Drug_1326,0.9579
2,ACH-000956,Drug_490,0.4130
3,ACH-000956,Drug_558,0.8004
4,ACH-000956,Drug_195,0.5743


In [19]:
valid_keep.head()

,improve_sample_id,improve_chem_id,auc
0,ACH-000956,Drug_895,0.5071
1,ACH-000956,Drug_614,0.6525
2,ACH-000956,Drug_1040,0.8944
3,ACH-000323,Drug_1279,0.8691
4,ACH-000323,Drug_685,0.8527


In [20]:
train_keep.shape, valid_keep.shape

((7616, 3), (952, 3))

In [21]:
train_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]
valid_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]

In [22]:
train_keep.head()

,Cell_Line,Drug_ID,AUC
0,ACH-000956,Drug_749,0.7153
1,ACH-000956,Drug_1326,0.9579
2,ACH-000956,Drug_490,0.4130
3,ACH-000956,Drug_558,0.8004
4,ACH-000956,Drug_195,0.5743


In [23]:
valid_keep.head()

,Cell_Line,Drug_ID,AUC
0,ACH-000956,Drug_895,0.5071
1,ACH-000956,Drug_614,0.6525
2,ACH-000956,Drug_1040,0.8944
3,ACH-000323,Drug_1279,0.8691
4,ACH-000323,Drug_685,0.8527


In [24]:
samp_drug = valid_keep["Drug_ID"].unique()[-1]
samp_ach = np.array(valid_keep["Cell_Line"].unique()[-1])

In [25]:
print(samp_drug)
print(samp_ach)

Drug_1326
ACH-000828


In [26]:
train_gcn_feats = []
train_adj_list = []
for drug_id in train_keep["Drug_ID"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_adj_mat[drug_id])

In [27]:
len(train_gcn_feats), len(train_adj_list)

(7616, 7616)

In [28]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in valid_keep["Drug_ID"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_adj_mat[drug_id])

In [29]:
len(valid_gcn_feats), len(valid_adj_list)

(952, 952)

In [30]:
len(valid_gcn_feats) + len(train_gcn_feats)

8568

In [31]:
%%time
# reduce the values to float16
train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")

CPU times: user 489 ms, sys: 802 ms, total: 1.29 s
Wall time: 1.29 s


In [32]:
train_gcn_feats.shape

(7616, 223, 75)

In [33]:
type(train_gcn_feats)

numpy.ndarray

In [34]:
train_adj_list.shape

(7616, 223, 223)

In [35]:
type(train_adj_list)

numpy.ndarray

In [36]:
train_keep["Cell_Line"].values.reshape(-1,1)

array([['ACH-000956'],
       ['ACH-000956'],
       ['ACH-000956'],
       ...,
       ['ACH-000828'],
       ['ACH-000828'],
       ['ACH-000828']], dtype=object)

In [37]:
train_keep["Cell_Line"].values.reshape(-1,1).shape

(7616, 1)

In [38]:
type(train_keep["Cell_Line"].values.reshape(-1,1))

numpy.ndarray

In [39]:
# valid_keep["Cell_Line"].values.reshape(-1,1)

In [40]:
valid_keep["Cell_Line"].values.reshape(-1,1).shape

(952, 1)

In [41]:
valid_keep["AUC"].shape

(952,)

In [42]:
valid_keep["AUC"].values.reshape(-1,1).shape

(952, 1)

In [43]:
type(valid_keep["AUC"].values.reshape(-1,1))

numpy.ndarray

In [44]:
# Okay, we cannot use the previous generator, therefore we will build a new custom generator that weill allow us to index correctly

In [45]:
# We do this using a class instance/object. We use __init__ to initialize the object's attributes - like 

In [46]:
# This say hello is called a method (eventhough it is a function) asit lives inside a class

In [47]:
import numpy as np
import tensorflow as tf

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, feature1, feature2, feature3, feature4, feature5, labels, batch_size=32, shuffle=True):
        """
        Initializes the data generator.

        :param feature1, feature2, feature3, feature4, feature5: The 5 different input features.
        :param labels: The target variable.
        :param batch_size: The size of the batch to generate.
        :param shuffle: Whether to shuffle data after each epoch.
        """
        self.feature1 = feature1
        self.feature2 = feature2
        self.feature3 = feature3
        self.feature4 = feature4
        self.feature5 = feature5
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.labels))
        self.on_epoch_end()

    def __len__(self):
        # Number of batches per epoch
        return int(np.ceil(len(self.labels) / self.batch_size))

    def on_epoch_end(self):
        # Shuffling the indexes after each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # Generate one batch of data
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Extract the individual features for the batch
        batch_feature1 = self.feature1[batch_indexes]
        batch_feature2 = self.feature2[batch_indexes]
        batch_feature3 = self.feature3[batch_indexes]
        batch_feature4 = self.feature4[batch_indexes]
        batch_feature5 = self.feature5[batch_indexes]
        
        # Get the corresponding labels for the batch
        batch_labels = self.labels[batch_indexes]

        # Return the individual features and labels
        return (batch_feature1, batch_feature2, batch_feature3, batch_feature4, batch_feature5), batch_labels

In [48]:
batch_size = 32

In [49]:
train_gen_alt = DataGenerator(train_gcn_feats, train_adj_list, train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["AUC"].values.reshape(-1,1), batch_size=32)

In [50]:
val_gen_alt = DataGenerator(valid_gcn_feats, valid_adj_list, valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["AUC"].values.reshape(-1,1), batch_size=32)

In [51]:
# cat = 0
# for X1, y in train_gen_alt:
#     print(len(X1), y.shape)
#     cat = cat + 1
#     print(cat)

In [52]:
# cat = 0
# for X1, y in val_gen_alt:
#     print(len(X1), y.shape)
#     cat = cat + 1
#     print(cat)

In [53]:
# I think we have the necessary data generators, maybe we need to use these to train the model

In [54]:
# Okay, we trained the model, let's see if we can get the trained model to predict the outcomes

In [55]:
# load the trained model

In [57]:
model = tf.keras.models.load_model("new_gen_results/DeepCDR_model/DeepCDR_model_with_data_generator")

In [58]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 223, 223)]   0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 223, 75)]    0           []                               
                                                                                                  
 dot (Dot)                      (None, 223, 75)      0           ['input_2[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 223, 256)     19456       ['dot[0][0]']                

In [59]:
# now for prediction 
def batch_predict(model, generator, steps, flatten=True, verbose=False):
    predictions = []
    true_values = []

    for batch_features, batch_labels in generator:
        # Get model predictions for the batch
        batch_predictions = model.predict(batch_features, batch_size=generator.batch_size, verbose=0)
        pred = batch_predictions.flatten()
        y = batch_labels.flatten()
        predictions.extend(pred)
        true_values.extend(y)
        # print("Batch Predict:")
    print(f"Predictions: {len(predictions)}")
    print(f"True: {len(true_values)}")
    return np.array(predictions), np.array(true_values)

In [63]:
generator_batch_size = 32

In [64]:
# get the predictions on the validation set
val_steps = int(np.ceil(len(valid_gcn_feats) / generator_batch_size))
val_steps

30

In [65]:
preds, true = batch_predict(model, val_gen_alt, val_steps)

2025-04-16 13:06:05.391884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


Predictions: 952
True: 952


In [67]:
preds.shape

(952,)

In [71]:
# preds

In [68]:
true.shape

(952,)

In [72]:
# true